***Hyperparameter Tuning Using Grid Search With Parallel Computing***

Sequential code adapted from: https://towardsdatascience.com/grid-search-in-python-from-scratch-hyperparameter-tuning-3cca8443727b

In [15]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
import time
import joblib
import threading
import multiprocessing
import os

**Generate dataset**

In [16]:
def generate_data(xmin,xmax,Delta,noise):
    # Calculate f=sin(x1)+cos(x2)
    x1 = np.arange(xmin,xmax+Delta,Delta)   # generate x1 values from xmin to xmax
    x2 = np.arange(xmin,xmax+Delta,Delta)   # generate x2 values from xmin to xmax
    x1, x2 = np.meshgrid(x1,x2)             # make x1,x2 grid of points
    f = np.sin(x1) + np.cos(x2)             # calculate for all (x1,x2) grid
    # Add random noise to f
    random.seed(2020)                       # set random seed for reproducibility
    for i in range(len(f)):
        for j in range(len(f[0])):
            f[i][j] = f[i][j] + random.uniform(-noise,noise)  # add random noise to f(x1,x2)

    # Calculate and print the number of records
    num_records = x1.size  # size of the grid (number of points)
    print(f"Number of records generated: {num_records}")
    return x1,x2,f


# Create {x1,x2,f} dataset every 1.0 from -10 to 10, with a noise of +/- 0.5
x1,x2,f=generate_data(-10,10,0.457,0.5) # Changed delta to generate different number of records

Number of records generated: 2025


**Prepare Data for Model Training**

In [17]:
def prepare_data(x1,x2,f):
    X = []
    for i in range(len(f)):
        for j in range(len(f)):
            X_term = []
            X_term.append(x1[i][j])
            X_term.append(x2[i][j])
            X.append(X_term)
    y=f.flatten()
    X=np.array(X)
    y=np.array(y)
    return X,y

# Prepare X and y
X,y = prepare_data(x1,x2,f)

**1. Kernel Ridge Regression (KRR) with Cross-Validation**

BASELINE

In [18]:
def KRR_function(X,y):
    # Initialize lists with final results
    y_pred_total = []
    y_test_total = []
    # Split data into test and train: random state fixed for reproducibility
    kf = KFold(n_splits=10,shuffle=True,random_state=2020)
    # kf-fold cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Scale X_train and X_test
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        # Fit KRR with (X_train_scaled, y_train), and predict X_test_scaled
        KRR = KernelRidge()
        y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        # Append y_pred and y_test values of this k-fold step to list with total values
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
    # Flatten lists with test and predicted values
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    # Calculate error metric of test and predicted values: rmse
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    return rmse
KRR_function(X,y)

1.0286383063881945

Hyperparameter Grid Search for KRR

In [19]:
def KRR_function(hyperparams,X,y):
    # Assign hyper-parameters
    alpha_value,gamma_value = hyperparams
    # Initialize lists with final results
    y_pred_total = []
    y_test_total = []
    # Split data into test and train: random state fixed for reproducibility
    kf = KFold(n_splits=10,shuffle=True,random_state=2020)
    # kf-fold cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Scale X_train and X_test
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        # Fit KRR with (X_train_scaled, y_train), and predict X_test_scaled
        KRR = KernelRidge(kernel='rbf',alpha=alpha_value,gamma=gamma_value)
        y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        # Append y_pred and y_test values of this k-fold step to list with total values
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
    # Flatten lists with test and predicted values
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    # Calculate error metric of test and predicted values: rmse
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    print('KRR k-fold cross-validation . alpha: %7.6f, gamma: %7.4f, RMSE: %7.4f' %(alpha_value,gamma_value,rmse))
    return rmse

Sequential

In [23]:
graph_x = []
graph_y = []
graph_z = []

min_rmse = 1.0286383063881945
optimal_alpha = 1
optimal_gamma = None

def create_hyperparams_grid(X,y):
    global min_rmse, optimal_alpha, optimal_gamma
    for alpha_value in np.arange(-5.0,2.0,0.7):
        alpha_value = pow(10,alpha_value)
        graph_x_row = []
        graph_y_row = []
        graph_z_row = []
        for gamma_value in np.arange(0.0,20,2):
            hyperparams = (alpha_value,gamma_value)
            rmse = KRR_function(hyperparams,X,y)
            if rmse < min_rmse:
                optimal_alpha = alpha_value
                optimal_gamma = gamma_value
                min_rmse = rmse
            graph_x_row.append(alpha_value)
            graph_y_row.append(gamma_value)
            graph_z_row.append(rmse)
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)


start = time.time()
create_hyperparams_grid(X,y)
end = time.time()
print("Execution time:", end-start)
print(min_rmse, optimal_alpha, optimal_gamma)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]
print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

KRR k-fold cross-validation . alpha: 0.000010, gamma:  0.0000, RMSE:  1.0297
KRR k-fold cross-validation . alpha: 0.000010, gamma:  2.0000, RMSE:  0.2995
KRR k-fold cross-validation . alpha: 0.000010, gamma:  4.0000, RMSE:  0.3171
KRR k-fold cross-validation . alpha: 0.000010, gamma:  6.0000, RMSE:  0.3338
KRR k-fold cross-validation . alpha: 0.000010, gamma:  8.0000, RMSE:  0.3491
KRR k-fold cross-validation . alpha: 0.000010, gamma: 10.0000, RMSE:  0.3667
KRR k-fold cross-validation . alpha: 0.000010, gamma: 12.0000, RMSE:  0.3837
KRR k-fold cross-validation . alpha: 0.000010, gamma: 14.0000, RMSE:  0.4066
KRR k-fold cross-validation . alpha: 0.000010, gamma: 16.0000, RMSE:  0.4366
KRR k-fold cross-validation . alpha: 0.000010, gamma: 18.0000, RMSE:  0.4718
KRR k-fold cross-validation . alpha: 0.000050, gamma:  0.0000, RMSE:  1.0297
KRR k-fold cross-validation . alpha: 0.000050, gamma:  2.0000, RMSE:  0.2969
KRR k-fold cross-validation . alpha: 0.000050, gamma:  4.0000, RMSE:  0.3111

Parallel

References:

-https://www.scaler.com/topics/python-parallel-for-loop/


-https://joblib.readthedocs.io/en/stable/parallel.html



Using multithreading with Joblib

In [24]:
lock = threading.Lock()
graph_x = []
graph_y = []
graph_z = []
ccc = 0
min_rmse = 1.0286383063881945
optimal_alpha = 1
optimal_gamma = None
alpha_values = np.arange(-5.0,2.0,0.7)
gamma_values = np.arange(0.0,20,2)


def func_for_alpha_loop(alpha_value):
    global ccc, min_rmse, optimal_alpha, optimal_gamma
    alpha_value = pow(10,alpha_value)
    graph_x_row = []
    graph_y_row = []
    graph_z_row = []
    for gamma_value in gamma_values:
        print(f"Task {ccc} is running in thread {threading.get_ident()}")
        # lock.acquire()
        ccc += 1
        # lock.release()
        hyperparams = (alpha_value,gamma_value)
        rmse = KRR_function(hyperparams,X,y)
        if rmse < min_rmse:
            optimal_alpha = alpha_value
            optimal_gamma = gamma_value
            min_rmse = rmse
        graph_x_row.append(alpha_value)
        graph_y_row.append(gamma_value)
        graph_z_row.append(rmse)
    graph_x.append(graph_x_row)
    graph_y.append(graph_y_row)
    graph_z.append(graph_z_row)


num_cores = joblib.cpu_count()
print(num_cores)
start = time.time()
joblib.Parallel(n_jobs=6, prefer="threads")(joblib.delayed(func_for_alpha_loop)(alpha_value) for alpha_value in alpha_values)
end = time.time()
print("Execution time:", end-start)
print("Value of ccc: ", ccc)
print(min_rmse, optimal_alpha, optimal_gamma)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]
print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

8
Task 0 is running in thread 6220722176
Task 1 is running in thread 6237548544
Task 2 is running in thread 6254374912
Task 3 is running in thread 6271201280
Task 4 is running in thread 6288027648
Task 5 is running in thread 6304854016
KRR k-fold cross-validation . alpha: 0.006310, gamma:  0.0000, RMSE:  1.0297
Task 6 is running in thread 6288027648
KRR k-fold cross-validation . alpha: 0.000251, gamma:  0.0000, RMSE:  1.0297
Task 7 is running in thread 6254374912
KRR k-fold cross-validation . alpha: 0.000050, gamma:  0.0000, RMSE:  1.0297
Task 8 is running in thread 6237548544
KRR k-fold cross-validation . alpha: 0.031623, gamma:  0.0000, RMSE:  1.0297
Task 9 is running in thread 6304854016
KRR k-fold cross-validation . alpha: 0.000010, gamma:  0.0000, RMSE:  1.0297
Task 10 is running in thread 6220722176
KRR k-fold cross-validation . alpha: 0.001259, gamma:  0.0000, RMSE:  1.0297
Task 11 is running in thread 6271201280
KRR k-fold cross-validation . alpha: 0.031623, gamma:  2.0000, RMS

Using multiprocessing with Joblib

In [25]:
manager = multiprocessing.Manager()
ccc = manager.Value('i', 0)
graph_x = manager.list()
graph_y = manager.list()
graph_z = manager.list()
min_rmse = manager.Value(float, 1.0286383063881945)
optimal_alpha = manager.Value(float, 1)
optimal_gamma = manager.Value(float, None)
lock = multiprocessing.Manager().Lock()

alpha_values = np.arange(-5.0,2.0,0.7)
gamma_values = np.arange(0.0,20,2)

def func_for_alpha_loop(alpha_value):  
    alpha_value = pow(10,alpha_value)
    graph_x_row = []
    graph_y_row = []
    graph_z_row = []
    for gamma_value in gamma_values:
        print(f"Task {ccc} is running in process {os.getpid()}")
        # lock.acquire()
        ccc.value += 1
        # lock.release()
        hyperparams = (alpha_value,gamma_value)
        rmse = KRR_function(hyperparams,X,y)
        if rmse < min_rmse.value:
            optimal_alpha.value = alpha_value
            optimal_gamma.value = gamma_value
            min_rmse.value = rmse
        graph_x_row.append(alpha_value)
        graph_y_row.append(gamma_value)
        graph_z_row.append(rmse)
    graph_x.append(graph_x_row)
    graph_y.append(graph_y_row)
    graph_z.append(graph_z_row)


num_cores = joblib.cpu_count()
print(num_cores)
start = time.time()
joblib.Parallel(n_jobs=10)(joblib.delayed(func_for_alpha_loop)(alpha_value) for alpha_value in alpha_values)
end = time.time()
print("Execution time:", end-start)
print("ccc: ", ccc.value)
print(min_rmse.value, optimal_alpha.value, optimal_gamma.value)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]
print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

8
Task Value('i', 0) is running in process 7105
Task Value('i', 0) is running in process 7101
Task Value('i', 1) is running in process 7106
Task Value('i', 2) is running in process 7108
Task Value('i', 3) is running in process 7102
Task Value('i', 4) is running in process 7103
Task Value('i', 5) is running in process 7109
Task Value('i', 6) is running in process 7100
Task Value('i', 7) is running in process 7104
Task Value('i', 8) is running in process 7107
KRR k-fold cross-validation . alpha: 0.000251, gamma:  0.0000, RMSE:  1.0297
Task Value('i', 9) is running in process 7105
KRR k-fold cross-validation . alpha: 0.000050, gamma:  0.0000, RMSE:  1.0297
Task Value('i', 10) is running in process 7106
KRR k-fold cross-validation . alpha: 0.000010, gamma:  0.0000, RMSE:  1.0297
Task Value('i', 11) is running in process 7101
KRR k-fold cross-validation . alpha: 3.981072, gamma:  0.0000, RMSE:  1.0297
Task Value('i', 12) is running in process 7102
KRR k-fold cross-validation . alpha: 0.1584

Profiling

In [ ]:
%timeit -n 4 -r 1 create_hyperparams_grid(X, y)

In [ ]:
import cProfile
cProfile.run('create_hyperparams_grid(X, y)')

In [ ]:
!pip install line_profiler

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f create_hyperparams_grid create_hyperparams_grid(X, y)
